In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 44.8 MB/s 
     |████████████████████████████████| 182 kB 55.3 MB/s 


In [1]:
from transformers import pipeline
import pandas as pd

In [ ]:
!unzip Food\ Small.zip

Archive:  Food Small.zip
  inflating: nutrients_csvfile.csv   


In [ ]:
df = pd.read_csv("nutrients_csvfile.csv")

In [ ]:
df.head()

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
0,Cows' milk,1 qt.,976,660,32,40,36,0,48,Dairy products
1,Milk skim,1 qt.,984,360,36,t,t,0,52,Dairy products
2,Buttermilk,1 cup,246,127,9,5,4,0,13,Dairy products
3,"Evaporated, undiluted",1 cup,252,345,16,20,18,0,24,Dairy products
4,Fortified milk,6 cups,"1,419","1,373",89,42,23,1.4,119,Dairy products


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Food      335 non-null    object
 1   Measure   335 non-null    object
 2   Grams     335 non-null    object
 3   Calories  334 non-null    object
 4   Protein   335 non-null    object
 5   Fat       335 non-null    object
 6   Sat.Fat   333 non-null    object
 7   Fiber     335 non-null    object
 8   Carbs     335 non-null    object
 9   Category  335 non-null    object
dtypes: object(10)
memory usage: 26.3+ KB


In [ ]:
df.dropna(inplace=True)

In [ ]:
def clean_data(x):
  if '.' in x:
    return x
  if("," in x):
    x = x.replace("," , "")
    if x.isnumeric():
      return x
  else:
    ls = x.split(' ')
    if(ls[0].isnumeric()):
      return ls[0]
    else:
      return str(-1)

In [ ]:
df['Measure'] = df['Measure'].apply(lambda x: clean_data(x))
df['Grams'] = df['Grams'].apply(lambda x: clean_data(x))
df['Calories'] = df['Calories'].apply(lambda x: clean_data(x))
df['Protein'] = df['Protein'].apply(lambda x: clean_data(x))
df['Fat'] = df['Fat'].apply(lambda x: clean_data(x))
df['Sat.Fat'] = df['Sat.Fat'].apply(lambda x: clean_data(x))
df['Fiber'] = df['Fiber'].apply(lambda x: clean_data(x))
df['Carbs'] = df['Carbs'].apply(lambda x: clean_data(x))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 332 entries, 0 to 334
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Food      332 non-null    object
 1   Measure   332 non-null    object
 2   Grams     332 non-null    object
 3   Calories  332 non-null    object
 4   Protein   332 non-null    object
 5   Fat       332 non-null    object
 6   Sat.Fat   332 non-null    object
 7   Fiber     332 non-null    object
 8   Carbs     332 non-null    object
 9   Category  332 non-null    object
dtypes: object(10)
memory usage: 28.5+ KB


In [ ]:
features = df.columns

In [ ]:
df.shape

(332, 10)

In [ ]:
tqa = pipeline(task="table-question-answering", model="google/tapas-large-finetuned-wtq")

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/490 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/154 [00:00<?, ?B/s]

In [ ]:
df.head()

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
0,Cows' milk,1 qt.,976,660,32,40,36,0,48,Dairy products
1,Milk skim,1 qt.,984,360,36,-1,-1,0,52,Dairy products
2,Buttermilk,1,246,127,9,5,4,0,13,Dairy products
3,"Evaporated, undiluted",1,252,345,16,20,18,0,24,Dairy products
4,Fortified milk,6,1419,1373,89,42,23,1.4,119,Dairy products


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neulab/omnitab-large")
model = AutoModelForSeq2SeqLM.from_pretrained("neulab/omnitab-large")

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/506k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/965 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/813M [00:00<?, ?B/s]

In [ ]:
question = "What Food item has 620 Grams and 10 Protein"
encoding = tokenizer(table=df[50:25], query=question, return_tensors="pt")
outputs = model.generate(**encoding)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

[' ground lean']


In [ ]:
df.shape

(332, 10)

In [ ]:
def predict_responses(df, query):
  responses = []
  try:
    prev_i = 0
    for i in range(25, 333, 25):
      encoding = tokenizer(table=df[prev_i:i], query=query, return_tensors="pt")
      outputs = model.generate(**encoding)
      responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True))
      prev_i = i
  except:
    pass
  return responses

In [ ]:
question = "What Food item has 120 Calories and 35 Carbs"
predict_responses(df, question)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 1024 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[[' cream or half-and-half'],
 [' yolks'],
 [' corned beef hash canned'],
 [' Salmon'],
 [' mushroom canned'],
 [' split cooked peas'],
 [' apple juice canned'],
 [' fresh, raw'],
 [' watermelon'],
 [' corn grits cooked'],
 [' chocolate fudge'],
 [' honey'],
 [' table (12.2% alcohol)']]

In [ ]:
from transformers import AutoModelForTableQuestionAnswering, AutoTokenizer, pipeline

# Load model & tokenizer
tapas_model = AutoModelForTableQuestionAnswering.from_pretrained('navteca/tapas-large-finetuned-wtq')
tapas_tokenizer = AutoTokenizer.from_pretrained('navteca/tapas-large-finetuned-wtq')

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/582 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/262k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/194 [00:00<?, ?B/s]

In [ ]:
nlp = pipeline('table-question-answering', model=tapas_model, tokenizer=tapas_tokenizer)

## Final Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
import random

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neulab/omnitab-large")
model = AutoModelForSeq2SeqLM.from_pretrained("neulab/omnitab-large")

In [ ]:
def predict_responses(df, query):
  responses = []
  try:
    prev_i = 0
    for i in range(25, 333, 25):
      encoding = tokenizer(table=df[prev_i:i], query=query, return_tensors="pt")
      outputs = model.generate(**encoding)
      responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True))
      prev_i = i
  except:
    pass
  return responses

In [ ]:
df.tail()

,Food,Measure,Grams,Calories,Protein,Fat,Sat.Fat,Fiber,Carbs,Category
330,Fruit-flavored soda,12 oz.,346,161,0,0,0,0,42,"Drinks,Alcohol, Beverages"
331,Ginger ale,12 oz.,346,105,0,0,0,0,28,"Drinks,Alcohol, Beverages"
332,Root beer,12 oz.,346,140,0,0,0,0,35,"Drinks,Alcohol, Beverages"
333,Coffee,1,230,3,-1,0,0,0,1,"Drinks,Alcohol, Beverages"
334,Tea,1,230,4,0,-1,0,0,1,"Drinks,Alcohol, Beverages"


In [ ]:
question = "What Food item in which Measure has 140 Calories, 0 Protein and 35 Carbs"
predict_responses(df, question)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 1024 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[[' milk'],
 [' lard'],
 [' chicken livers, fried'],
 [' Lima'],
 [' mustard greens'],
 [' squash'],
 [' nectar, or juice'],
 [' fresh, raw'],
 [' raw, diced'],
 [' macaroni'],
 [' chocolate fudge'],
 [' custard'],
 [' wines']]

### Approach 2: (Show any one product instead)

In [7]:
random.choice(list(range(25, 333, 25)))
list(range(25, 333, 25))

[25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325]

In [8]:
def predict_responses(df, query):
    responses = []
    try:
        start_idx = random.choice(list(range(25, 333, 25)))
        encoding = tokenizer(table=df[start_idx:start_idx+25], query=query, return_tensors="pt")
        outputs = model.generate(**encoding)
        responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True))
    except:
        pass
    return responses